In [8]:
import pandas as pd
from arcgis import GIS
gis = GIS()
import requests
from datetime import datetime
import arcpy

bRedownloadLatest = False # will download feature class from agol, otherwise will use last download
bExportFeatureClass = False # export feature class
bClearFlags = False # will clear all flag fields from backup
bClearOverrides = False # will clear all override fields from backup

In [9]:
# Prep flag categories

# flag name must be short enough to be in dbf column name
dfFlags = pd.DataFrame([
    ['FL_REV'   , 'Not reviewed'                          , "([NOTES]=='') & ([NOTES_FURREV]=='')"],
    ['FL_LTPRV' , 'Less than previous forecast year'      , "(([MF2023]+[ADJ2023])<([M2019]+[aadtAdjFactor])) | (([MF2028]+[ADJ2028])<([MF2023]+[ADJ2023])) | (([MF2032]+[ADJ2032])<([MF2028]+[ADJ2028])) | (([MF2042]+[ADJ2042])<([MF2032]+[ADJ2032])) | (([MF2050]+[ADJ2050])<([MF2042]+[ADJ2042]))"],
    ['FL_ZERO'  , 'Zero volume'                           , "(([MF2023]+[ADJ2023])<=0) | (([MF2028]+[ADJ2028])<=0) | (([MF2032]+[ADJ2032])<=0) | (([MF2042]+[ADJ2042])<=0) | (([MF2050]+[ADJ2050])<=0)"],
    ['FL_HIADJ' , 'Model Adjustment Factor >95% base year', "((([aadtAdjFactor]/([M2019]+[aadtAdjFactor]))>0.95) | (([aadtAdjFactor]/([M2019]+[aadtAdjFactor]))<-0.95))"],
    ['FL_50LT19', 'HPMS: 2050 less than 2019'             , "(([MF2050]+[ADJ2050])<([M2019]+[aadtAdjFactor]))"],
    ['FL_503X19', 'HPMS: 2050 more than three times 2019' , "(([MF2050]+[ADJ2050])>(3*([M2019]+[aadtAdjFactor])))"],
    ['FL_FURREV', 'Further Review'                        , "([NOTES_FURREV]!='')"]
], columns=('flagName','flagDescription','flagCriteria'))

dfFlags.to_json('_site/data/flags.json', orient='records')

In [10]:
# convert csvs to jsons
lstSegidFiles = ['aadt', 'linear-forecasts', 'model-forecasts', 'previous-forecasts']
lstNonSegidFiles = ['aadt-sources', 'previous-forecasts-sources', 'projection-groups']

for file in lstSegidFiles + lstNonSegidFiles:
    # Read CSV file
    pd.read_csv('intermediate/' + file + '.csv').to_json('_site/data/' + file + '.json', orient='records')

In [11]:
fnSegments = 'data/segments/Segments_WF - 2023-08-01.shp'

seg_cols = ['SEGID','CO_NAME','PLANAREA','SHAPE'] ############## ADD SUBAREA ID WHEN FILE HAS IT #####################


# USE ALL PLANAREAS since this app will be for entire state
# filter by PLANAREA in segments shapefile
#filterPlanArea = ['WFRC','MAG'] # must be an array... if only single item, the still include []

In [12]:
if bRedownloadLatest:

    # Define the URL
    url = "https://services1.arcgis.com/taguadKoI1XFwivx/arcgis/rest/services/Forecasts_gdb/FeatureServer/0/query"

    # Base parameters for the request
    params = {
        "f": "json",
        "where": "1=1",
        "outFields": "*",
        "returnGeometry": "false",  # Assuming you only want attributes
    }

    all_features = []

    # Pagination parameters
    batch_size = 2000
    offset = 0

    while True:
        # Adjust parameters for pagination
        params['resultOffset'] = offset
        params['resultRecordCount'] = batch_size

        # Send the request
        response = requests.get(url, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()
            features = data.get('features', [])
            
            # Check if no features were returned, which means we're done
            if not features:
                break
            
            # Append features to the master list
            all_features.extend(features)
            
            # Increase the offset for next iteration
            offset += batch_size

        else:
            print("Error fetching data:", response.status_code)
            break

    # Extract attribute table
    attribute_table = [feature['attributes'] for feature in all_features]

    # Convert to DataFrame
    df = pd.DataFrame(attribute_table)

    # Generate a filename with the current timestamp in the specified folder
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f"backup\\forecastsegments_backup_{timestamp}.csv"

    # Save the DataFrame to a CSV file
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")
else:
    print("didn't download... will use latest")

didn't download... will use latest


In [13]:
# Define the URL
url = "https://services1.arcgis.com/taguadKoI1XFwivx/arcgis/rest/services/forecasts_logfile/FeatureServer/0/query"

# Base parameters for the request
params = {
    "f": "json",
    "where": "1=1",
    "outFields": "*",
    "returnGeometry": "false",  # If it's a stand-alone table, geometry isn't required.
    "outSR": "4326",  # Specify output spatial reference if needed
    "returnExceededLimitFeatures": "true"
}

all_features = []

# Pagination parameters
batch_size = 2000
offset = 0

while True:
    # Adjust parameters for pagination
    params['resultOffset'] = offset
    params['resultRecordCount'] = batch_size

    # Send the request
    response = requests.post(url, params=params)  # Use POST since some servers may have GET length limits

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        features = data.get('features', [])
        
        # Check if no features were returned, which means we're done
        if not features:
            break
        
        # Append features to the master list
        all_features.extend(features)
        
        # Increase the offset for next iteration
        offset += batch_size

    else:
        print("Error fetching data:", response.status_code)
        break

# Extract attribute table
attribute_table = [feature['attributes'] for feature in all_features]

# Convert to DataFrame
df = pd.DataFrame(attribute_table)

# Generate a filename with the current timestamp in the specified folder
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f"backup\\logfile_backup_{timestamp}.csv"

# Save the DataFrame to a CSV file
df.to_csv(filename, index=False)
print(f"Data saved to {filename}")

Data saved to backup\logfile_backup_20230907_102222.csv


In [14]:
#import datetime
#
## empty log file for AGOL
#dfLogFile = pd.DataFrame([
#    ['dummy','dummy',0,0,0,0,0,'dummy',datetime.datetime.now()]
#],columns=('SEGID','EDITKEY','ADJ2023','ADJ2028','ADJ2032','ADJ2042','ADJ2050','NOTES','TIMESTAMP'))
#
#dfLogFile.to_csv('results/forecasts-logfile.csv')
#
#dfLogFile

In [15]:
# Get data from last backup

import os

# List all backup files in the directory
backup_dir = "backup\\"
all_files = [f for f in os.listdir(backup_dir) if f.startswith('forecastsegments_backup_') and f.endswith('.csv')]

# Sort the files based on their timestamps
sorted_files = sorted(all_files, reverse=True)  # Latest timestamp will be first

# Check if there are any backup files
if sorted_files:
    latest_file = os.path.join(backup_dir, sorted_files[0])
    # Load the most recent backup into a dataframe
    dfLatest = pd.read_csv(latest_file)

    print(f"Loaded {latest_file} into a DataFrame")

    if bClearFlags:
        # Specify columns to delete
        columns_to_delete = [col for col in dfLatest.columns if col.startswith('FL_') or col.startswith('OV_')]

        # Remove the specified columns from df_filtered
        dfLatest.drop(columns_to_delete, axis=1, inplace=True)
    
        print(f"Dropped FL_ and OV_ columns")

else:
    dfLatest = pd.DataFrame()
    print("No backup files found in the directory.")

# You can then use dfLatest as your DataFrame


Loaded backup\forecastsegments_backup_20230906_182940.csv into a DataFrame


In [16]:
# read in segments, filter, and select only key columns
sdfSegments = pd.DataFrame.spatial.from_featureclass(fnSegments)
#sdfSegments = sdfSegments[sdfSegments['PLANAREA'].isin(filterPlanArea)]
sdfSegments = sdfSegments[seg_cols]

display(sdfSegments)

,SEGID,CO_NAME,PLANAREA,SHAPE
0,0006_000.0,MILLARD,UDOT,"{""paths"": [[[236177.7000000002, 4327541.25], [..."
1,0006_000.7,MILLARD,UDOT,"{""paths"": [[[237241.1799999997, 4327399.720000..."
2,0006_016.0,MILLARD,UDOT,"{""paths"": [[[261403.90000000037, 4327045.4], [..."
3,0006_046.0,MILLARD,UDOT,"{""paths"": [[[305380.16000000015, 4325741.85], ..."
4,0006_060.2,MILLARD,UDOT,"{""paths"": [[[324618, 4337936.9], [324647.40000..."
...,...,...,...,...
8722,UTA_7132,,WFRC,"{""paths"": [[[424262.6891235444, 4508606.236747..."
8723,UTA_7316,,WFRC,"{""paths"": [[[420773.38413784996, 4495314.95905..."
8724,UTA_7320,,WFRC,"{""paths"": [[[419033.24434054515, 4494684.05276..."
8725,UTA_7328,,WFRC,"{""paths"": [[[415002.73054989125, 4492771.26989..."


In [24]:
sdfSegments[sdfSegments['SEGID']=='1438_000.9']

,SEGID,CO_NAME,PLANAREA,SHAPE
3768,1438_000.9,DAVIS,WFRC,"{""paths"": [[[422188.71999999974, 4539109.4], [..."
8600,1438_000.9,DAVIS,WFRC,"{""paths"": [[[422140.650591542, 4538901.3089648..."


In [17]:
# export segment JSON

# manually modify one without plan area
sdfSegments.loc[sdfSegments['SEGID']=='2322_004.5', 'PLANAREA'] = 'WFRC'

sdfSegments[[x for x in seg_cols if x != 'SHAPE']].to_json('_site/data/segments.json', orient='records')

In [18]:
# read in intermediate data
dfModVolAdj = pd.read_csv('intermediate/model-forecasts.csv')
dfModVolAdj['YEAR'] = dfModVolAdj['YEAR'].astype(str)
dfModVolAdj['YEAR'] = 'M' + dfModVolAdj['YEAR']

dfModVolAdj_pvModAadt = dfModVolAdj.pivot_table(index=['SEGID','FAC_WDAVG','aadtAdjFactor'], columns='YEAR', values='modAadt')
dfModVolAdj_pvModAadt.reset_index(inplace=True)

dfModVolAdj = pd.read_csv('intermediate/model-forecasts.csv')
dfModVolAdj['YEAR'] = dfModVolAdj['YEAR'].astype(str)
dfModVolAdj['YEAR'] = 'MF' + dfModVolAdj['YEAR']

dfModVolAdj_pvModAadtAdj = dfModVolAdj.pivot_table(index=['SEGID','FAC_WDAVG','aadtAdjFactor'], columns='YEAR', values='modForecast')
dfModVolAdj_pvModAadtAdj.reset_index(inplace=True)

dfModVolAdj = pd.DataFrame.merge(dfModVolAdj_pvModAadt, dfModVolAdj_pvModAadtAdj, on=('SEGID','FAC_WDAVG','aadtAdjFactor'))

dfModVolAdj

YEAR,SEGID,FAC_WDAVG,aadtAdjFactor,M2019,M2023,M2028,M2032,M2042,M2050,MF2019,MF2023,MF2028,MF2032,MF2042,MF2050
0,0006_146.9,0.9840,-18.0,1535.0,1671.0,1738.0,1892.0,2120.0,2409.0,1500.0,1700.0,1700.0,1900.0,2100.0,2400.0
1,0006_149.9,0.9840,856.0,1585.0,1799.0,2012.0,2075.0,2652.0,5357.0,2400.0,2700.0,2900.0,2900.0,3500.0,6200.0
2,0006_150.6,0.9840,657.0,1784.0,1989.0,2209.0,2281.0,2891.0,5598.0,2400.0,2600.0,2900.0,2900.0,3500.0,6300.0
3,0006_152.6,0.9840,-119.0,2536.0,2719.0,2988.0,3080.0,3756.0,6225.0,2400.0,2600.0,2900.0,3000.0,3600.0,6100.0
4,0006_152.9,0.9593,-124.0,3883.0,4230.0,4700.0,4991.0,5675.0,8072.0,3800.0,4100.0,4600.0,4900.0,5600.0,7900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3940,WFRC_8261,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3941,WFRC_8263,1.0924,0.0,4245.0,5503.0,6511.0,6667.0,6888.0,7125.0,4200.0,5500.0,6500.0,6700.0,6900.0,7100.0
3942,WFRC_8264,1.0924,0.0,NaN,NaN,5737.0,7598.0,9690.0,11846.0,NaN,NaN,5700.0,7600.0,9700.0,12000.0
3943,WFRC_8265,1.0924,0.0,NaN,1383.0,1736.0,2664.0,3380.0,3694.0,NaN,1400.0,1700.0,2700.0,3400.0,3700.0


In [19]:
# join segment data and forecast data
_sdf = pd.DataFrame.merge(dfModVolAdj, sdfSegments, on='SEGID', how='right')

# add columns for Adjustments
_sdf['ADJ2019'] = 0
_sdf['ADJ2023'] = 0
_sdf['ADJ2028'] = 0
_sdf['ADJ2032'] = 0
_sdf['ADJ2042'] = 0
_sdf['ADJ2050'] = 0
_sdf['ADJHIST'] = 0
_sdf['NOTES'] = ""
_sdf['NOTES_FURREV'] = ""

# replace with latest file
if dfLatest.empty:
    print("The latest dataframe is empty.")
else:
    print("The latest dataframe is not empty.")
        
    # Specify columns to include from latest download (FLAGS WILL NOT BE INCLUDED WHEN FILTERED PRIOR TO THIS STEP)
    columns_to_include = [col for col in dfLatest.columns if col.startswith('ADJ') or col.startswith('FL_') or col.startswith('OV_') or col == 'SEGID' or col == 'NOTES' or col == 'NOTES_FURREV']

    # Filter df_filtered to only have the specified columns
    dfLatest_filtered = dfLatest[columns_to_include]

    # Merge sdfData with df_filtered on SEGID
    _sdfWithLatest = _sdf.merge(dfLatest_filtered, on='SEGID', how='left', suffixes=('_delete', ''))

    # Drop any duplicate columns (those with '_delete' suffix) after the merge 
    for column in _sdfWithLatest.columns:
        if column.endswith('_delete'):
            _sdfWithLatest.drop(column, axis=1, inplace=True)

    _sdfWithLatest['NOTES'].fillna('0', inplace=True)
    _sdfWithLatest['NOTES'] = _sdfWithLatest['NOTES'].astype(str)
    _sdfWithLatest.loc[(_sdfWithLatest['NOTES']=='0'), 'NOTES'] = ''
    _sdfWithLatest['NOTES'] = _sdfWithLatest['NOTES'].str.strip()

    _sdfWithLatest['NOTES_FURREV'].fillna('0', inplace=True)
    _sdfWithLatest['NOTES_FURREV'] = _sdfWithLatest['NOTES_FURREV'].astype(str)
    _sdfWithLatest.loc[(_sdfWithLatest['NOTES_FURREV']=='0'), 'NOTES_FURREV'] = ''
    _sdfWithLatest['NOTES_FURREV'] = _sdfWithLatest['NOTES_FURREV'].str.strip()

# add flag columns
# Loop through the rows of 'dfFlags'
for index, row in dfFlags.iterrows():
    # Get the flag name from the 'flagName' column
    flag_name = row['flagName']

    # Get the flag criteria from the 'flagCriteria' column (if needed)
    flag_criteria = row['flagCriteria']

    # Add a new column to 'dfSegs' with the flag name and set it equal to evaluated criteria, prepare criteria string with replace
    expression = "_sdfWithLatest['" + flag_name + "'] = " + flag_criteria.replace("[", "_sdfWithLatest['").replace("]", "']")
    print(expression)

    # execute expression!
    exec(expression)
    #print(result)  # Output will be 11
    
    if bClearFlags:
        display('flags overwritten')
        # add override column
        _sdfWithLatest[flag_name.replace('FL_')] = 0
    if bClearOverrides: 
        display('overrides overwritten')
        _sdfWithLatest[flag_name.replace('OV_')] = 0

_sdfWithLatest.fillna(0,inplace=True)

sdfSegmentsWData = _sdfWithLatest
sdfSegmentsWData

The latest dataframe is not empty.
_sdfWithLatest['FL_REV'] = (_sdfWithLatest['NOTES']=='') & (_sdfWithLatest['NOTES_FURREV']=='')
_sdfWithLatest['FL_LTPRV'] = ((_sdfWithLatest['MF2023']+_sdfWithLatest['ADJ2023'])<(_sdfWithLatest['M2019']+_sdfWithLatest['aadtAdjFactor'])) | ((_sdfWithLatest['MF2028']+_sdfWithLatest['ADJ2028'])<(_sdfWithLatest['MF2023']+_sdfWithLatest['ADJ2023'])) | ((_sdfWithLatest['MF2032']+_sdfWithLatest['ADJ2032'])<(_sdfWithLatest['MF2028']+_sdfWithLatest['ADJ2028'])) | ((_sdfWithLatest['MF2042']+_sdfWithLatest['ADJ2042'])<(_sdfWithLatest['MF2032']+_sdfWithLatest['ADJ2032'])) | ((_sdfWithLatest['MF2050']+_sdfWithLatest['ADJ2050'])<(_sdfWithLatest['MF2042']+_sdfWithLatest['ADJ2042']))
_sdfWithLatest['FL_ZERO'] = ((_sdfWithLatest['MF2023']+_sdfWithLatest['ADJ2023'])<=0) | ((_sdfWithLatest['MF2028']+_sdfWithLatest['ADJ2028'])<=0) | ((_sdfWithLatest['MF2032']+_sdfWithLatest['ADJ2032'])<=0) | ((_sdfWithLatest['MF2042']+_sdfWithLatest['ADJ2042'])<=0) | ((_sdfWithLatest['M

,SEGID,FAC_WDAVG,aadtAdjFactor,M2019,M2023,M2028,M2032,M2042,M2050,MF2019,...,FL_HIADJ,OV_HIADJ,FL_50LT19,OV_50LT19,FL_503X19,OV_503X19,FL_FURREV,OV_FURREV,NOTES,NOTES_FURREV
0,0006_000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,0,False,0,False,0,False,0,,
1,0006_000.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,0,False,0,False,0,False,0,,
2,0006_016.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,0,False,0,False,0,False,0,,
3,0006_046.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,0,False,0,False,0,False,0,,
4,0006_060.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,0,False,0,False,0,False,0,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30983,UTA_7132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,0,False,0,False,0,False,0,,
30984,UTA_7316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,0,False,0,False,0,False,0,,
30985,UTA_7320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,0,False,0,False,0,False,0,,
30986,UTA_7328,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,0,False,0,False,0,False,0,,


In [20]:
sdfSegmentsWData[['SEGID','NOTES_FURREV','FL_FURREV','OV_FURREV']]

,SEGID,NOTES_FURREV,FL_FURREV,OV_FURREV
0,0006_000.0,,False,0
1,0006_000.7,,False,0
2,0006_016.0,,False,0
3,0006_046.0,,False,0
4,0006_060.2,,False,0
...,...,...,...,...
30983,UTA_7132,,False,0
30984,UTA_7316,,False,0
30985,UTA_7320,,False,0
30986,UTA_7328,,False,0


In [21]:
# export
#if bShapefileExport: #sdfSegmentsWData.spatial.to_featureclass('results/ForecastSegments/ForecastSegments.shp',sanitize_columns=False)
from arcgis.features import GeoAccessor

if bExportFeatureClass:
    # Define the geodatabase path
    gdb_path        = r"results\Forecasts.gdb"
    gdb_path_backup = r"backup\Forecasts_backup.gdb"

    # Check if the geodatabase exists, if not, create it
    if not arcpy.Exists(gdb_path):
        arcpy.CreateFileGDB_management(r"results", "Forecasts.gdb")
        
    # Check if the geodatabase exists, if not, create it
    if not arcpy.Exists(gdb_path_backup):
        arcpy.CreateFileGDB_management(r"backup", "Forecasts_backup.gdb")

    # Export SDF to the geodatabase as a feature class
    feature_class_name = "ForecastSegments"

    # Generate a filename with the current timestamp in the specified folder
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    feature_class_name_with_timestamp = f"ForecastSegments_{timestamp}"

    # file to replace
    sdfSegmentsWData.spatial.to_featureclass(location=f"{gdb_path_backup}\\{feature_class_name_with_timestamp}", sanitize_columns=False)
    print(f"Feature class {feature_class_name_with_timestamp} created in {gdb_path_backup}")

    # backup file
    sdfSegmentsWData.spatial.to_featureclass(location=f"{gdb_path}\\{feature_class_name}", sanitize_columns=False)

    print(f"Feature class {feature_class_name} created in {gdb_path}")

else:
    print ('not exported')


not exported


In [23]:
sdfSegmentsWData[sdfSegmentsWData['SEGID']=='1438_000.9']

,SEGID,FAC_WDAVG,aadtAdjFactor,M2019,M2023,M2028,M2032,M2042,M2050,MF2019,...,FL_HIADJ,OV_HIADJ,FL_50LT19,OV_50LT19,FL_503X19,OV_503X19,FL_FURREV,OV_FURREV,NOTES,NOTES_FURREV
3769,1438_000.9,1.0924,3066.0,5442.0,5701.0,6964.0,1253.0,1266.0,6427.0,8500.0,...,False,0,False,0,False,0,False,0,,
3770,1438_000.9,1.0924,3066.0,5442.0,5701.0,6964.0,1253.0,1266.0,6427.0,8500.0,...,False,0,False,0,False,0,False,0,,
3771,1438_000.9,1.0924,3066.0,5442.0,5701.0,6964.0,1253.0,1266.0,6427.0,8500.0,...,False,0,False,0,False,0,False,0,,
3772,1438_000.9,1.0924,3066.0,5442.0,5701.0,6964.0,1253.0,1266.0,6427.0,8500.0,...,False,0,False,0,False,0,False,0,,
3773,1438_000.9,1.0924,3066.0,5442.0,5701.0,6964.0,1253.0,1266.0,6427.0,8500.0,...,False,0,False,0,False,0,False,0,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8912,1438_000.9,1.0924,3066.0,5442.0,5701.0,6964.0,1253.0,1266.0,6427.0,8500.0,...,False,0,False,0,False,0,False,0,,
8913,1438_000.9,1.0924,3066.0,5442.0,5701.0,6964.0,1253.0,1266.0,6427.0,8500.0,...,False,0,False,0,False,0,False,0,,
8914,1438_000.9,1.0924,3066.0,5442.0,5701.0,6964.0,1253.0,1266.0,6427.0,8500.0,...,False,0,False,0,False,0,False,0,,
8915,1438_000.9,1.0924,3066.0,5442.0,5701.0,6964.0,1253.0,1266.0,6427.0,8500.0,...,False,0,False,0,False,0,False,0,,


In [28]:
duplicated_segid = sdfSegmentsWData[sdfSegmentsWData.duplicated('SEGID', keep=False)]['SEGID'].unique()
duplicates.unique()

array(['0013_000.0', '1438_000.9', '2040_007.2', '2094_002.5',
       '2107_003.5', 'UTA_7564', 'UTA_7572', 'UTA_7576', 'UTA_7404',
       'UTA_7150', 'UTA_7262', 'UTA_7250', 'UTA_7300', 'UTA_7120',
       'UTA_7121', 'UTA_7221'], dtype=object)